In [2]:
import requests
import pandas as pd
import time
import re
import csv
import os
from bs4 import BeautifulSoup
import random
import pickle
from tqdm import tqdm
import numpy as np
import os

import sys
sys.path.append("..") # 親ディレクトリを追加
from module.random_agent import RandomUserAgent
from module.path_reader import PathReader

In [ ]:
# what: HTMLを解析してレース結果テーブルをDataFrame化する関数
# for:  AIモデルの入力形式に合わせる
# in:   取得したhtml(.bin)
# out:  レース結果テーブル(DataFrame)
def parse_race_html(bin_path):
    with open(bin_path, "rb") as f:
            html_text = f.read().decode("EUC-JP", errors="ignore")
    soup = BeautifulSoup(html_text, "html.parser")
    result_table = soup.find("table", class_="RaceTable01")
    
    if not result_table:
        raise ValueError("レース結果テーブルが見つかりません。")

    rows = result_table.find_all("tr")[1:]  # ヘッダを除外
    race_data = []

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 15:
            continue

        # --- 馬IDを取得 ---
        horse_tag = row.find("a", href=re.compile(r"/horse/(\d+)"))
        horse_id = re.search(r"/horse/(\d+)", horse_tag["href"]).group(1) if horse_tag else None

        # --- 騎手IDを取得 ---
        jockey_tag = row.find("a", href=re.compile(r"/jockey/result/recent/(\d+)/"))
        jockey_id = re.search(r"/jockey/result/recent/(\d+)/", jockey_tag["href"]).group(1) if jockey_tag else None

        # --- 調教師IDを取得 ---
        trainer_tag = row.find("a", href=re.compile(r"/trainer/result/recent/(\d+)/"))
        trainer_id = re.search(r"/trainer/result/recent/(\d+)/", trainer_tag["href"]).group(1) if trainer_tag else None

        race_data.append([
            cols[0].get_text(strip=True),  # 着順
            cols[1].get_text(strip=True),  # 枠番
            cols[2].get_text(strip=True),  # 馬番
            horse_id,                      # 馬ID
            cols[4].get_text(strip=True),  # 性齢
            cols[5].get_text(strip=True),  # 斤量
            jockey_id,                     # 騎手ID
            cols[7].get_text(strip=True),  # タイム
            cols[8].get_text(strip=True),  # 着差
            cols[9].get_text(strip=True),  # 人気
            cols[10].get_text(strip=True), # オッズ
            cols[11].get_text(strip=True), # 後3F
            cols[12].get_text(strip=True), # 通過
            trainer_id,                    # 厩舎ID
            cols[14].get_text(strip=True), # 馬体重
        ])

    df = pd.DataFrame(race_data, columns=[
        "rank", "wakuban", "umaban", "horse_id", "sex&age", "weight_carried",
        "jockey_id", "time", "margin", "popularity", "odds", "last3f", "passing",
        "trainer_id", "body_weight"])
    # df = pd.DataFrame(race_data, columns=["着順", "馬名", "性齢", "斤量", "騎手", "タイム", "人気", "オッズ", "後3F", "厩舎", "馬体重"])
    return df

In [92]:
# what: HTMLを解析して払い戻しテーブルをDataFrame化する関数
# for:  AIモデルの入力形式に合わせる
# in:   取得したhtml(.bin)
# out:  払い戻しテーブル(DataFrame)
from bs4 import BeautifulSoup
import pandas as pd

def parse_return_html(html_text):
    soup = BeautifulSoup(html_text, "html.parser")

    pay_tables = soup.find_all("table", class_="Payout_Detail_Table")
    pay_data = []

    for tbl in pay_tables:
        for row in tbl.find_all("tr"):
            bet_type = row.find("th").get_text(strip=True) if row.find("th") else None
            result = " / ".join(span.get_text(strip=True) for span in row.select("td.Result span") if span.get_text(strip=True))
            payout = " / ".join(span.get_text(strip=True) for span in row.select("td.Payout span") if span.get_text(strip=True))
            popularity = " / ".join(span.get_text(strip=True) for span in row.select("td.Ninki span") if span.get_text(strip=True))

            pay_data.append([bet_type, result, payout, popularity])

    pay_df = pd.DataFrame(pay_data, columns=["券種", "馬番", "払戻金", "人気"])
    return pay_df

In [ ]:
# what: レース結果テーブルの前処理をする関数
# for:  AIモデルがうけつけられるようにする
# in:   レース結果テーブルの列(.pkl)
# out:  レース結果テーブルの列(.pkl)

def parse_sex_age(sexage):
    # 例: "牡4" -> ("牡", 4)
    # {牡:0, 牝:1, セ: 2, その他:np.nan}
    if pd.isna(sexage): return (np.nan, np.nan)

    # --- 性別(sex) ---
    sex_char = str(sexage[0])
    if sex_char == "牡":
        sex = 0
    elif sex_char == "牝":
        sex = 1
    elif sex_char == "セ":   # 騙馬（せん馬）
        sex = 2
    else:
        sex = np.nan

    # --- 年齢(age) ---
    try:
        age = int(sexage[1:])
    except:
        age = np.nan
    return sex, age

def parse_bodyweight(bw):
    # "494(-4)" -> weight=494, diff=-4
    try:
        s = str(bw)
        if "(" in s:
            w = int(s.split("(")[0])
            diff = int(s.split("(")[1].rstrip(")"))
        else:
            w = int(s)
            diff = np.nan
        return w, diff
    except:
        return (np.nan, np.nan)
    
def time_to_seconds(tstr):
    # "1:51.3" -> seconds float
    try:
        if pd.isna(tstr): return np.nan
        if ":" in str(tstr):
            mm, ss = str(tstr).split(":")
            return int(mm) * 60 + float(ss)
        else:
            return float(tstr)
    except:
        return np.nan

## 実行関数：HTMLの取得とレース結果・払い戻しテーブルの作成

In [ ]:
# 入力パラメータ

# 実行環境(NotePC/Desktop)の選択
# reader = PathReader("../file_path_NotePC.json") # NotePC用
reader = PathReader("../file_path_Desktop.json") # Desktop用

# スクレイピングするrace_id_liseの選択
race_id_list_name = "race_id_list_2301_2404.csv"

In [ ]:
# 入力データと出力データの絶対パス生成

# race_idが保存されているCSVファイルのパス
race_id_list_path = os.path.join(reader.get_path("data_folder"), race_id_list_name)
# 保存フォルダのパス
save_dir = os.path.join(reader.get_path("data_folder"), "race_result_html")

### HTMLの取得

In [ ]:
# CSVの読み込み
df = pd.read_csv(race_id_list_path)
ua = RandomUserAgent() # ランダムなUser-Agentを取得

for race_id in tqdm(df["race_id"], total=len(df)):
    race_id = str(race_id)
    url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"

    # ファイル保存パス 
    save_path = os.path.join(save_dir, f"{race_id}.bin")
    if os.path.exists(save_path):
        continue # 既にそのrace_idが取得済みならスキップ
    else:
        try:
            res = requests.get(url, headers={"User-Agent": ua.user_agent}, timeout=10)
            res.raise_for_status()  # エラーがあれば例外を発生

            # HTMLをバイナリで保存
            with open(save_path, "wb") as f:
                f.write(res.content)

            # アクセス間隔を少し空ける（サーバー負荷対策）
            time.sleep(random.uniform(0.8, 2.0))

        except Exception as e:
            print(f"Error fetching {race_id}: {e}")

100%|██████████| 2064/2064 [19:28<00:00,  1.77it/s] 


### レース結果テーブルの作成
pickleファイルを展開
→race_idを取得
→race_idが重複する場合はスキップ

In [ ]:
# what: 各レースのbinファイルからレース結果を抽出し、1つのテーブルに結合しpickleで保存する関数
# for:  特徴量の抽出用
# in:   取得したrace_id_list(.csv)とhtml(.bin)
# out:  結合されたresult_table(.pickle)
result_table_path = os.path.join(reader.get_path("data_folder"), "race_result_table.pkl")
bin_dir = os.path.join(reader.get_path("data_folder"), "race_result_html")
df = pd.read_csv(race_id_list_path, dtype={"race_id": str, "date": str})
race_to_date = dict(zip(df["race_id"], df["date"])) # race_id -> data の辞書を作成

# 既存pickleのrace_idを確認
existing_df = pd.read_pickle(result_table_path)
existing_ids = set(existing_df["race_id"].astype(str))

# 新しく解析するrace_idだけを抽出
target_ids = [str(rid) for rid in df["race_id"] if str(rid) not in existing_ids]

new_dfs = []
for race_id in tqdm(target_ids, total=len(target_ids)):
    bin_path = os.path.join(bin_dir, f"{race_id}.bin")
    if not os.path.exists(bin_path):
        print(f"Missing bin file: {race_id}")
        continue
    try:
        # --- HTML解析 ---
        df_race = parse_race_html(bin_path)
        df_race.insert(0, "race_id", race_id)
        df_race.insert(1, "date", race_to_date[race_id]) # race_idと対応する日付を挿入
        new_dfs.append(df_race)
    except Exception as e:
        print(f"Error fetching {race_id}: {e}")
new_result_df = pd.concat(new_dfs, ignore_index=True)

# 追加するテーブルに前処理をしておく
new_result_df.insert(2, "is_win", (new_result_df["rank"].astype(str).str.strip() == "1").astype(int)) # ターゲットエンコーディング(1着のみ1)
new_result_df[["sex","age"]] = new_result_df["sex&age"].apply(lambda x: pd.Series(parse_sex_age(x)))
new_result_df["age"] = new_result_df["age"].astype(float)
new_result_df["weight_carried"] = pd.to_numeric(new_result_df["weight_carried"], errors="coerce")
new_result_df["time_sec"] = new_result_df["time"].apply(time_to_seconds)
new_result_df["last3f"] = pd.to_numeric(new_result_df["last3f"], errors="coerce")
new_result_df[["body_weight","body_diff"]] = new_result_df["body_weight"].apply(lambda x: pd.Series(parse_bodyweight(x)))
new_result_df["odds"] = pd.to_numeric(new_result_df["odds"], errors="coerce")
new_result_df["popularity"] = pd.to_numeric(new_result_df["popularity"], errors="coerce")
new_result_df = new_result_df.drop(columns=["sex&age", "time"])

result_df = pd.concat([existing_df, new_result_df], ignore_index=True)
print(f"✅ 新規{len(new_result_df)}件を追加しました（合計 {len(result_df)} 件）")
print(result_df)
result_table = result_df.to_pickle(result_table_path)

100%|██████████| 5/5 [00:00<00:00, 28.08it/s]


✅ 新規83件を追加しました（合計 28506 件）
           date       race_id  is_win rank    horse_id  weight_carried  \
0      20231001  202309040910       1    1  2019105746            58.0   
1      20231001  202309040910       0    2  2019102983            58.0   
2      20231001  202309040910       0    3  2018104746            58.0   
3      20231001  202309040910       0    4  2019100108            58.0   
4      20231001  202309040910       0    5  2019103518            58.0   
...         ...           ...     ...  ...         ...             ...   
28501  20240428  202408030411       0   14  2018104609            58.0   
28502  20240428  202408030411       0   15  2020103650            58.0   
28503  20240428  202408030411       0   16  2016104946            58.0   
28504  20240428  202408030411       0   中止  2019100630            58.0   
28505  20240428  202408030411       0   取消  2016104851            58.0   

      jockey_id  popularity   odds  last3f trainer_id  body_weight sex  age  \
0    

### pickleファイルの確認

In [ ]:
# ファイルパス
result_table_path = r"C:\Users\yasak\Desktop\mykeibaAI_ver1p0\data\race_result_table.pkl"

# pickleファイルを読み込む
df = pd.read_pickle(result_table_path)
# df
# 下10件を表示
print(df.tail(30))


           date       race_id  is_win rank    horse_id  weight_carried  \
28393  20240428  202405020403       0   13  2021103705            57.0   
28394  20240428  202405020403       0   14  2021106574            57.0   
28395  20240428  202405020403       0   15  2021100425            56.0   
28396  20240428  202405020403       0   16  2021103492            57.0   
28397  20240428  202408030404       1    1  2021104427            55.0   
28398  20240428  202408030404       0    2  2021105002            57.0   
28399  20240428  202408030404       0    3  2021104924            57.0   
28400  20240428  202408030404       0    4  2021110022            57.0   
28401  20240428  202408030404       0    5  2021102701            54.0   
28402  20240428  202408030404       0    6  2021103570            57.0   
28403  20240428  202408030404       0    7  2021106937            57.0   
28404  20240428  202408030404       0    8  2021101771            54.0   
28405  20240428  202408030404       0 

<p>回収率シミュレーションのときにparse_return_html関数は使う</p>

In [ ]:
with open(r"C:\Users\yasak\Desktop\mykeibaAI_ver1p0\result.bin", "rb") as f:
    html_text = f.read().decode("EUC-JP", errors="ignore")

# new_result_df_result = parse_race_html(html_text)
# print(df_result)

# print("----------------------------------------------------------------")
df_pay = parse_return_html(html_text)
print(df_pay)

    着順         馬名  性齢    斤量   騎手     タイム  人気    オッズ   後3F     厩舎      馬体重
0    1  ホウオウルーレット  牡4  58.0  岩田康  1:51.3   5    7.2  35.9   美浦栗田  494(-4)
1    2  キュールエフウジン  牡4  58.0  藤岡佑  1:51.4   8   22.2  36.3   栗東中尾  494(-6)
2    3  セイクリッドゲイズ  セ5  58.0  岩田望  1:51.5   4    7.0  36.8  栗東佐々木   494(0)
3    4  プリモスペランツァ  牡4  58.0  鮫島駿  1:51.6  11   33.9  37.1   栗東中竹   494(0)
4    5   マルブツプライド  牡4  58.0   川須  1:51.6  10   30.1  36.8   栗東加用  532(+8)
5    6     カズプレスト  牡4  58.0   亀田  1:51.7   6   10.4  37.6  栗東高柳大  522(-6)
6    7   ホウオウフウジン  牡4  58.0    幸  1:52.0   9   25.3  38.0   栗東矢作  524(-6)
7    8      クロニクル  牡4  58.0  吉田隼  1:52.1   7   13.1  37.8  栗東田中克  522(+6)
8    9   ルイナールカズマ  牡4  58.0  藤岡康  1:52.1  12  124.1  36.5  栗東奥村豊  496(-6)
9   10   ラインオブソウル  牡4  58.0   松若  1:52.3   3    6.9  37.5   栗東音無  520(+4)
10  11   タガノエスコート  牡4  58.0  和田竜  1:52.7   1    3.1  38.2   栗東小林  500(-6)
11  12     ロコポルティ  牡5  58.0  Ｍデム  1:52.7   2    5.6  38.0  栗東西園正  526(-2)
--------------------------------------